In [29]:
import pandas as pd
import numpy as np
import math
import boto3
from io import BytesIO
import os
from datetime import datetime, timedelta

pd.set_option('display.max_columns', None)

Pushing all local fils to S3

In [3]:
client = boto3.client("s3")
names = [
    r"\\ant\dept-eu\Amazon-Flex-Europe\EU-OE\LPnS\LPnS_Model\LPnS_Wave_Optimizer_V4.xlsm",                               # LPNS Wave Plan 
    r"\\ant.amazon.com\dept-eu\Amazon-Flex-Europe\Data\OE\AMZL\UTR Model\UTRChangeLog.xlsx",                             # UTR Buffers
    r"\\ant\dept-eu\TBA\UK\Business Analyses\CentralOPS\Scheduling\UK\FlexData\SPR Planner3.xlsx",                       # SPRs
    r"\\ant\dept-eu\TBA\UK\Business Analyses\CentralOPS\Scheduling\UK\FlexData\UK_Flex_Schedule_Ahead_Percentage.xlsm",  # SA Table
    r"\\ant\dept-eu\Amazon-Flex-Europe\EU-OE\LPnS\ETL\UK_AMZL_Flex_SCMS_Attributes_21d.txt",                             # SCMS
    r"\\ant\dept-eu\Amazon-Flex-Europe\EU-OE\LPnS\ETL\UK_CVP_Plan_14d.txt",                                              # CVP Data + 2W SnOP Forecast
    r"\\ant\dept-eu\Amazon-Flex-Europe\EU-OE\LPnS\ETL\UK_CVP_Plan_All.txt",                                              # CVP Data + SnOP Forecast
    r"\\ant\dept-eu\Amazon-Flex-Europe\EU-OE\LPnS\ETL\UK_AMZL_Flex_SCMS_Attributes_120d.txt",    
    r"\\ant\dept-eu\Amazon-Flex-Europe\EU-OE\LPnS\ETL\UK_AMZL_Route_Summary_SPR.txt",                                    # SPR Historicals
    r"\\ant\dept-eu\Amazon-Flex-Europe\EU-OE\LPnS\ETL\UK_AMZL_Route_Planning_Agg_Pivot.txt",
    r"\\ant\dept-eu\Amazon-Flex-Europe\EU-OE\LPnS\ETL\UK_AMZL_RoBL_Data_For_UTR_Mod.txt",
    r"\\ant\dept-eu\Amazon-Flex-Europe\EU-OE\LPnS\ETL\UK_AMZL_Provider_Demand.txt",
    r"\\ant\dept-eu\Amazon-Flex-Europe\EU-OE\LPnS\ETL\UK_Siphon_Data_Pivot.txt",
    r"\\ant\dept-eu\Amazon-Flex-Europe\EU-OE\LPnS\ETL\UK_AMZL_Routing_DSP_to_Flex.txt",
    r"\\ant\dept-eu\Amazon-Flex-Europe\EU-OE\LPnS\ETL\UK_AMZL_Routing_DSP_to_Flex_21d.txt",
    r"\\ant\dept-eu\Amazon-Flex-Europe\EU-OE\LPnS\ETL\UK_AMZL_Date_Station_Cycle.txt",
    r"\\ant\dept-eu\Amazon-Flex-Europe\EU-OE\LPnS\ETL\UK_AMZL_Flex_Fill_At_Sequence.txt",
    r"\\ant\dept-eu\Amazon-Flex-Europe\EU-OE\LPnS\ETL\UK_AMZL_DEA_PM_Data_Grp_Pivot.txt",                                # DEA Data
    r"\\ant\dept-eu\Amazon-Flex-Europe\EU-OE\LPnS\ETL\UK_AMZL_RoBL_Data_For_RLD_Mod.txt",                                # RoBL Data for UTR + Reporting
    r"\\ant\dept-eu\Amazon-Flex-Europe\EU-OE\LPnS\ETL\UK_Flex_Block_Actuals_by_Cycle.txt",                               # Historical Blocks
    r"\\ant\dept-eu\Amazon-Flex-Europe\EU-OE\LPnS\ETL\UK_AMZL_CO_Volume_Forecast.txt",                                   # CO 48H Forecast ND
    r"\\ant\dept-eu\EUCentralOPS\Volume-Management\Control Tower\Forecast\EU_SameDay\logs\SD_forecaster_daily_logs.csv"  # CO 48H Forecast SD
]


for file in names:
    buffer = BytesIO()
    fmt = os.path.splitext(file)[1].lower()
    name = os.path.splitext(os.path.basename(file))[0]

    if fmt == ".txt":
        df = pd.read_csv(file, sep="\t")
        df.to_parquet(buffer, index=False)
        save_fmt = ".parquet"
    elif fmt == ".csv":
        df = pd.read_csv(file)
        df.to_parquet(buffer, index=False)
        save_fmt = ".parquet"
    elif fmt in [".xlsx", ".xlsm", ".xls"]:
        save_fmt = fmt
        with open(file, "rb") as f:
            buffer.write(f.read())
    
    buffer.seek(0)

    client.put_object(
        Bucket= "uk-flex-scheduling",
        Key = f"{name}{save_fmt}",
        Body = buffer.getvalue(),
        ContentType="application/octet-stream"
        )

    print(f"✅ Uploaded {name}")

✅ Uploaded LPnS_Wave_Optimizer_V4
✅ Uploaded UTRChangeLog
✅ Uploaded SPR Planner3
✅ Uploaded UK_Flex_Schedule_Ahead_Percentage
✅ Uploaded UK_AMZL_Flex_SCMS_Attributes_21d
✅ Uploaded UK_CVP_Plan_14d
✅ Uploaded UK_CVP_Plan_All
✅ Uploaded UK_AMZL_Flex_SCMS_Attributes_120d
✅ Uploaded UK_AMZL_Route_Summary_SPR
✅ Uploaded UK_AMZL_Route_Planning_Agg_Pivot
✅ Uploaded UK_AMZL_RoBL_Data_For_UTR_Mod
✅ Uploaded UK_AMZL_Provider_Demand
✅ Uploaded UK_Siphon_Data_Pivot
✅ Uploaded UK_AMZL_Routing_DSP_to_Flex
✅ Uploaded UK_AMZL_Routing_DSP_to_Flex_21d
✅ Uploaded UK_AMZL_Date_Station_Cycle
✅ Uploaded UK_AMZL_Flex_Fill_At_Sequence
✅ Uploaded UK_AMZL_DEA_PM_Data_Grp_Pivot
✅ Uploaded UK_AMZL_RoBL_Data_For_RLD_Mod
✅ Uploaded UK_Flex_Block_Actuals_by_Cycle
✅ Uploaded UK_AMZL_CO_Volume_Forecast
✅ Uploaded SD_forecaster_daily_logs


S3 Downloader

In [4]:
s3_client = boto3.client("s3")

BUCKET = "uk-flex-scheduling" 

def get_s3_object(bucket, prefix, header=0, sheet_name=0):
    try:
        response = s3_client.list_objects_v2(Bucket=bucket, Prefix=prefix)
        contents = response.get('Contents', [])
        
        if len(contents) >= 1:
            # Sort by last modified and get the most recent file
            latest_file = sorted(contents, key=lambda x: x['LastModified'], reverse=True)[0]
            key = latest_file['Key']
            
            # Skip if it's just a directory
            if key.endswith('/'):
                raise ValueError(f"No files found in directory: {key}")
                
            obj = s3_client.get_object(Bucket=bucket, Key=key)
            data = obj['Body'].read()
            
            if key.lower().endswith('.parquet'):
                return pd.read_parquet(BytesIO(data))
            elif key.lower().endswith('.csv'):
                return pd.read_csv(BytesIO(data))
            elif key.lower().endswith(('.xlsx', '.xls', '.xlsm')):
                return pd.read_excel(BytesIO(data), header=header, sheet_name=sheet_name)
            elif key.lower().endswith('.txt'):
                return pd.read_csv(BytesIO(data), sep='\t', encoding='utf-8')
        else:
            raise ValueError("No objects found in the prefix")
            
    except Exception as e:
        print(f"Error reading from S3: {str(e)}")
        raise

Get SNOP FCST out

In [5]:

df = get_s3_object(BUCKET, "UK_CVP_Plan_14d.parquet")

cols = ["joinkey", "week", "ofd_date", "station", "cycle", "value", "flex_share_cvp", "flex_spr_cvp", "source"]

df["ofd_volume"] = df["ofd_volume"].astype(int)
df["week"] = pd.to_datetime(df["ofd_date"]).dt.isocalendar().week
df["flex_spr"] = df["flex_spr"].astype(int, errors="ignore")
df["flex_routes_available"] = df["flex_routes_available"].astype(int, errors="ignore")
df["source"] = "snop_fcst"
df = df.rename(columns={
    "flex_share": "flex_share_cvp",
    "flex_spr": "flex_spr_cvp",
    "ofd_volume": "value"
})

df["joinkey"] = df["ofd_date"].astype(str) + df["station"] + df["cycle"]

snop_fcst = df[cols]
snop_fcst.head()

,joinkey,week,ofd_date,station,cycle,value,flex_share_cvp,flex_spr_cvp,source
0,2025-06-16VCY2CYCLE_SD_C,25,2025-06-16,VCY2,CYCLE_SD_C,0,1.00000,20.0,snop_fcst
1,2025-05-29VSA6CYCLE_SD_C,22,2025-05-29,VSA6,CYCLE_SD_C,1991,1.00000,20.0,snop_fcst
2,2025-06-10DWR1CYCLE_1,24,2025-06-10,DWR1,CYCLE_1,56784,0.03998,45.0,snop_fcst
3,2025-05-23DHA2CYCLE_SD_D,21,2025-05-23,DHA2,CYCLE_SD_D,1695,1.00000,21.0,snop_fcst
4,2025-06-19DCR3CYCLE_SD_D,25,2025-06-19,DCR3,CYCLE_SD_D,930,1.00000,24.0,snop_fcst


Get CO ND Forecast out

In [6]:

df = get_s3_object(BUCKET, "UK_AMZL_CO_Volume_Forecast.parquet")

cols = ["joinkey", "ofd_date", "station", "cycle", "value", "source"]

df["published"] = pd.to_datetime(df["published"])
df = df.loc[df.groupby(["ofddate", "stationcode", "wavegroupname"])["published"].idxmax()]
df["source"] = "co_fcst"
df = df.rename(columns={
    "ofddate": "ofd_date",
    "stationcode": "station",
    "wavegroupname": "cycle",
    "volume": "value"
})

df["joinkey"] = df["ofd_date"] + df["station"] + df["cycle"]

co_nd_fcst = df[cols]
co_nd_fcst.head()

,joinkey,ofd_date,station,cycle,value,source
101,2025-05-30DAB1CYCLE_1,2025-05-30,DAB1,CYCLE_1,28036,co_fcst
115,2025-05-30DBH3CYCLE_1,2025-05-30,DBH3,CYCLE_1,44633,co_fcst
184,2025-05-30DBI2CYCLE_1,2025-05-30,DBI2,CYCLE_1,48270,co_fcst
33,2025-05-30DBI4CYCLE_1,2025-05-30,DBI4,CYCLE_1,29556,co_fcst
180,2025-05-30DBI5CYCLE_1,2025-05-30,DBI5,CYCLE_1,38008,co_fcst


Get CO SD Forecast out

In [7]:

df = get_s3_object(BUCKET, "SD_forecaster_daily_logs.parquet")

cols = ["joinkey", "ofd_date", "station", "cycle", "value", "source"]

# Rename columns first
df = df.rename(columns={
    "Node": "station",
    "OFD_DATE": "ofd_date",
    "Expected OFD ": "value"
})

# Filter for UK
df = df[df["Country"] == "UK"]

# Filter for today and tomorrow
today = datetime.today().date()
tomorrow = today + timedelta(days=1)
df = df[df["ofd_date"].isin([today.strftime("%Y-%m-%d"), tomorrow.strftime("%Y-%m-%d")])]

# Cycle naming
df["cycle"] = "CYCLE_SD_" + df["Cycle"]

# Joining Key

df["joinkey"] = df["ofd_date"] + df["station"] + df["cycle"]

df["source"] = "co_fcst"
co_sd_fcst = df[cols]


Join all the tables together

In [8]:
forecasts = pd.concat(
    [snop_fcst, co_nd_fcst, co_sd_fcst],
    axis=0,
    ignore_index=True,
    sort=False
)

# Separate Column for SNOP FCST (2W, for weely SA)
cond_1 = forecasts["source"] == "snop_fcst"
forecasts["snop_fcst"] = forecasts["value"][cond_1]

# Separate Column for CO FCST (48H, for D-2 SA)
cond_2 = forecasts["source"] == "co_fcst"
forecasts["co_fcst"] = forecasts["value"][cond_2]

# Ensuring each row has its Flex Share and SPR
forecasts = forecasts.drop(["flex_share_cvp", "flex_spr_cvp"], axis=1)
forecasts = forecasts.merge(
    snop_fcst[["joinkey", "flex_share_cvp", "flex_spr_cvp"]],
    how="left",
    on="joinkey"
)

forecasts["source"].value_counts()
forecasts[cond_2].head()


,joinkey,week,ofd_date,station,cycle,value,source,snop_fcst,co_fcst,flex_share_cvp,flex_spr_cvp
6488,2025-05-30DAB1CYCLE_1,<NA>,2025-05-30,DAB1,CYCLE_1,28036,co_fcst,NaN,28036.0,0.040,48.0
6489,2025-05-30DBH3CYCLE_1,<NA>,2025-05-30,DBH3,CYCLE_1,44633,co_fcst,NaN,44633.0,0.035,48.0
6490,2025-05-30DBI2CYCLE_1,<NA>,2025-05-30,DBI2,CYCLE_1,48270,co_fcst,NaN,48270.0,0.030,48.0
6491,2025-05-30DBI4CYCLE_1,<NA>,2025-05-30,DBI4,CYCLE_1,29556,co_fcst,NaN,29556.0,0.035,45.0
6492,2025-05-30DBI5CYCLE_1,<NA>,2025-05-30,DBI5,CYCLE_1,38008,co_fcst,NaN,38008.0,0.035,48.0


SCMS Data

In [9]:

df = get_s3_object(BUCKET, "UK_AMZL_Flex_SCMS_Attributes_21d.parquet")

cols = ["joinkey", "week", "day", "ofd_date", "station", "cycle", "wave_capacity", "wave_start_time", "wave_frequency", "wave_end_time", "valid_from", "wave_max"]

df = df.rename(columns={
    "start_time": "wave_start_time",
    "end_time": "wave_end_time",
    "max_wave_capacity": "wave_capacity"
})

df["week"] = pd.to_datetime(df["ofd_date"]).dt.isocalendar().week
df["wave_start_time"] = pd.to_datetime(df["wave_start_time"], format="%H:%M").dt.strftime("%H:%M")
df["wave_frequency"] = pd.to_timedelta(df["wave_frequency"]).dt.total_seconds()/60
df["wave_end_time"] = pd.to_datetime(df["wave_end_time"], format="%H:%M").dt.strftime("%H:%M")
df["valid_from"] = pd.to_datetime(df["valid_from"], format="%Y-%m-%d").dt.strftime("%Y-%m-%d")

df["wave_minutes"] = (pd.to_datetime(df["wave_end_time"], format="%H:%M") - pd.to_datetime(df["wave_start_time"], format="%H:%M")).dt.total_seconds()/60
df["wave_minutes"] = df["wave_minutes"].astype(int)

df["max_wave_count"] = (df["wave_minutes"] / df["wave_frequency"]).astype(int, errors="ignore")
max_wave_cond = df["max_wave_count"] > 15
df["wave_max"] = df["max_wave_count"]
df.loc[max_wave_cond, "wave_max"] = 15

df["joinkey"] = df["ofd_date"] + df["station"] + df["cycle"]
df = df[cols]

scms_data = df.copy()

scms_data.head()

,joinkey,week,day,ofd_date,station,cycle,wave_capacity,wave_start_time,wave_frequency,wave_end_time,valid_from,wave_max
0,2025-06-17DXE1CYCLE_1,25,Tue,2025-06-17,DXE1,CYCLE_1,26.5,00:00,15.0,16:00,2025-04-05,15.0
1,2025-06-09DIG1CYCLE_SD_D,24,Mon,2025-06-09,DIG1,CYCLE_SD_D,40.0,19:30,15.0,20:00,2025-04-23,2.0
2,2025-06-02DME4CYCLE_SD_C,23,Mon,2025-06-02,DME4,CYCLE_SD_C,42.0,17:30,15.0,19:15,2025-04-02,7.0
3,2025-06-18DCR3CYCLE_SD_C,25,Wed,2025-06-18,DCR3,CYCLE_SD_C,28.0,17:30,15.0,19:15,2025-06-01,7.0
4,2025-06-17DXM4CYCLE_1,25,Tue,2025-06-17,DXM4,CYCLE_1,20.0,13:00,15.0,18:00,2025-04-05,15.0


UTR Data

In [10]:

df = get_s3_object(BUCKET, "UTRChangeLog.xlsx", sheet_name="DailyRange")

df = df.rename(columns={
    "JoinKey": "joinkey",
    "Buffer": "utr_buffer",
    "OFDDate": "ofd_date",
    "Station": "station",
    "Cycle": "cycle"
    })

df["ofd_date"] = pd.to_datetime(df["ofd_date"], format="%Y-%m-%d")
df["joinkey"] = df["ofd_date"].astype(str) + df["station"] + df["cycle"]
df = df[df["ofd_date"].between(pd.Timestamp.today().normalize(), pd.Timestamp.today().normalize() + pd.Timedelta(days=21))]

utr_data = df.copy()

utr_data.head()


,joinkey,ofd_date,station,cycle,utr_buffer
150,2025-05-31DAB1CYCLE_1,2025-05-31,DAB1,CYCLE_1,0
151,2025-06-01DAB1CYCLE_1,2025-06-01,DAB1,CYCLE_1,0
152,2025-06-02DAB1CYCLE_1,2025-06-02,DAB1,CYCLE_1,0
153,2025-06-03DAB1CYCLE_1,2025-06-03,DAB1,CYCLE_1,0
154,2025-06-04DAB1CYCLE_1,2025-06-04,DAB1,CYCLE_1,0


Calculated SPR Data

In [11]:

df = get_s3_object(BUCKET, "SPR Planner3.xlsx", header=1, sheet_name="Data")

cols = ["cycle", "station", "Calc SPR"]

df = df[cols]
df = df.rename(columns={
    "Calc SPR": "calc_spr"
})

spr_data = df.copy()

spr_data.head()

C:\Users\jklas\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


,cycle,station,calc_spr
0,CYCLE_1,DAB1,48
1,CYCLE_1,DBH3,48
2,CYCLE_1,DBI2,48
3,CYCLE_1,DBI4,45
4,CYCLE_1,DBI5,48


SA% Table

In [12]:
df = get_s3_object(BUCKET, "UK_Flex_Schedule_Ahead_Percentage.xlsm")

df = df.rename(columns={
    "DOW": "day",
    "Station": "station",
    "Cycle": "cycle",
    "SA% EOA": "sa%_eoa",
    "SA% WK": "sa%_week",
    "SA% D3": "sa%_d3",
    "SA% D2": "sa%_d2",
    "SA% D1": "sa%_d1",
    "SA% D0": "sa%_d0"
})
df = df.fillna(0)
sa_table = df.copy()

sa_table.head()

,day,station,cycle,sa%_eoa,sa%_week,sa%_d3,sa%_d2,sa%_d1,sa%_d0
0,Sun,DAB1,CYCLE_1,0.9,0.45,0.50,0.7,0.85,0.0
1,Mon,DAB1,CYCLE_1,0.9,0.45,0.50,0.7,0.85,0.0
2,Tue,DAB1,CYCLE_1,0.9,0.40,0.45,0.6,0.75,0.0
3,Wed,DAB1,CYCLE_1,0.9,0.45,0.50,0.7,0.85,0.0
4,Thu,DAB1,CYCLE_1,0.9,0.45,0.50,0.7,0.85,0.0


Get current Wave Plan

In [13]:

df = get_s3_object(BUCKET, "LPnS_Wave_Optimizer_V4.xlsm", header=1, sheet_name="WavePlanSD")

columns = [
    "Date", "Station", "Cycle",
    "D-1", "D-2", "D-3", "D-4", "D-5", "D-6", "D-7", "D-8", "D-9", "D-10", "D-11", "D-12", "D-13", "D-14", "D-15",
    "C-1", "C-2", "C-3", "C-4", "C-5", "C-6", "C-7", "C-8", "C-9", "C-10", "C-11", "C-12", "C-13", "C-14", "C-15"
    ]

df = df[columns]
df.columns = [col.strip().replace(" ", "_").replace("-", "_") for col in df.columns]

df = df.rename(columns={
    "Date": "ofd_date",
    "Station": "station",
    "Cycle": "cycle"
})

df["joinkey"] = df["ofd_date"].astype(str) + df["station"] + df["cycle"]
df = df.fillna(0)

wave_plan = df.copy()

wave_plan.head()



,ofd_date,station,cycle,D_1,D_2,D_3,D_4,D_5,D_6,D_7,D_8,D_9,D_10,D_11,D_12,D_13,D_14,D_15,C_1,C_2,C_3,C_4,C_5,C_6,C_7,C_8,C_9,C_10,C_11,C_12,C_13,C_14,C_15,joinkey
0,2025-06-01,DAB1,CYCLE_1,3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.5,0.0,0.0,0.0,0.0,0.0,0.0,15,15,15.0,15.0,15.0,15.0,15.0,15.0,15.0,0.0,0.0,0.0,0.0,0.0,0.0,2025-06-01DAB1CYCLE_1
1,2025-06-02,DAB1,CYCLE_1,3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.5,0.0,0.0,0.0,0.0,0.0,0.0,15,15,15.0,15.0,15.0,15.0,15.0,15.0,15.0,0.0,0.0,0.0,0.0,0.0,0.0,2025-06-02DAB1CYCLE_1
2,2025-06-03,DAB1,CYCLE_1,3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.5,0.0,0.0,0.0,0.0,0.0,0.0,15,15,15.0,15.0,15.0,15.0,15.0,15.0,15.0,0.0,0.0,0.0,0.0,0.0,0.0,2025-06-03DAB1CYCLE_1
3,2025-06-04,DAB1,CYCLE_1,3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.5,0.0,0.0,0.0,0.0,0.0,0.0,15,15,15.0,15.0,15.0,15.0,15.0,15.0,15.0,0.0,0.0,0.0,0.0,0.0,0.0,2025-06-04DAB1CYCLE_1
4,2025-06-05,DAB1,CYCLE_1,3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.5,0.0,0.0,0.0,0.0,0.0,0.0,15,15,15.0,15.0,15.0,15.0,15.0,15.0,15.0,0.0,0.0,0.0,0.0,0.0,0.0,2025-06-05DAB1CYCLE_1


Join it all together

In [ ]:

# Get all columns in theri original order
cols = list(dict.fromkeys(list(scms_data.columns) + list(forecasts.columns) + list(utr_data.columns) + list(spr_data.columns) + list(sa_table.columns) + list(wave_plan.columns)))

# Start with SCMS data and join Forecasts

df = scms_data.merge( # Join SCSMS + Forecasts
    forecasts,
    on="joinkey",
    how="left",
    suffixes=('', '_a')
) \
    .merge(  # Join UTR buffers
    utr_data,
    on="joinkey",
    how="left",
    suffixes=('', '_b')
) \
    .merge( # Join SPR Plan
    spr_data,
    on=["cycle", "station"],
    how="left",
    suffixes=('', '_c')
) \
    .merge( # Join SA Table
    sa_table,
    on=["day", "station", "cycle"],
    how="left",
    suffixes=('', '_d')
) \
    .merge( # Join Wave Plan
    wave_plan,
    on="joinkey",
    how="left",
    suffixes=('', '_e')
)

# Drop unnecesary cols Sort by Date/Cycle/Station
df = df.sort_values(["ofd_date", "cycle", "station"], ascending=False)

# Only include current Week and next week
cw = datetime.now().isocalendar().week
nw = cw + 1
relevant_weeks = df["week"].isin([cw, nw])
df = df[relevant_weeks]


# CALCULATED FIELDS

# Waves Available
wave_cols = ["D_1", "D_2", "D_3", "D_4", "D_5", "D_6", "D_7", "D_8", "D_9", "D_10", "D_11", "D_12", "D_13", "D_14", "D_15"]
df["waves_available"] = (df[wave_cols] > 0).sum(axis=1)

# Final Forecast = If CO Fcst notnull(): CO Fcst, else: SnOP FCST 
df["final_forecast"] = np.where((df["co_fcst"].isnull()==False), df["co_fcst"], df["snop_fcst"])

# Flex Volume = Final Forecast * Flex Share in CVP
df["flex_volume"] = df["final_forecast"] * df["flex_share_cvp"]

# Block Demand = Flex Volume / SPR
df["block_demand"] = df["flex_volume"] / df["flex_spr_cvp"]

# Wave Demand = Block Demand / Yard Capacity
df["wave_demand"] = df["block_demand"] / df["wave_capacity"]

# Seq mins
df["seq_mins"] = df["wave_max"]*60 - df["utr_buffer"]

# Inf Block Length = if Flex Volume =0: return 0, else: if "CYCLE_SD": ...

# Inf Seq Mins 
nd_group = df["cycle"].isin(["CYCLE_1", "CYCLE_2"])
sd_group = ~df["cycle"].isin(["CYCLE_1", "CYCLE_2"])
positive_demand = df["wave_demand"] > 0

df["infinity_seq_mins"] = np.nan
df["infinity_seq_mins"][nd_group] = df["seq_mins"] 
df["infinity_seq_mins"][~positive_demand] = np.nan
df["infinity_seq_mins"][~sd_group] = np.nan
df["infinity_seq_mins"][sd_group & positive_demand] = np.nan
df["max_pct"] = df[["sa%_d1", "sa%_d0"]].max(axis=1)                        #TODO: Think of importing the sa log table as a separate log file

def _calc_sd_infinity_mins(row):   
    wd = float(row["wave_demand"])
    wa = float(row["waves_available"])
    utr = float(row["utr_buffer"])
    max_pct = float(row["max_pct"])

    if pd.isna(wd) or pd.isna(max_pct):
        return np.nan
    
    primary_index = int(math.ceil(wd * max_pct))

    if 1 <= primary_index <= 15:
        pm = row.get(f"D_{primary_index}", 0.0)
        primary_minutes = 0.0 if pd.isna(pm) else float(pm)
    else:
        primary_minutes = 0.0

    # 3) Multiply by 60, subtract UTR → primary_result
    primary_result = primary_minutes * 60 - utr

    # 4) If primary_result < 0, do fallback using waves_available
    if primary_result < 0:
        fallback_index = int(math.ceil(wa * max_pct))
        if 1 <= fallback_index <= 15:
            fm = row.get(f"D_{fallback_index}", 0.0)
            fallback_minutes = 0.0 if pd.isna(fm) else float(fm)
        else:
            fallback_minutes = 0.0

        return fallback_minutes * 60 - utr

    # 5) Otherwise, just use primary_result
    return primary_result

sd_positive_mask = sd_group & positive_demand
df.loc[sd_positive_mask, "infinity_seq_mins"] = (
    df.loc[sd_positive_mask].apply(_calc_sd_infinity_mins, axis=1)
)
df = df.drop(columns=["max_pct"])

# DEA Cut-off
df["dea_cut_off"] = np.nan
# DEA Buffer Start = DEA Cut-off - DEA Buffer (usually 30 min)
df["dea_buffer_start"] = np.nan
# Max Block Length = (Infinity Seq + UTR) / 60
# Inf Block Length = if Flex Volume =0: return 0, else: if "CYCLE_SD": ...
df["infinity_block_length"] = np.nan

# Sertvice type
df["service_type"] = np.nan
df["service_type"][nd_group] = "AmFlex Car+"
df["service_type"][sd_group] = "AmFlex Same Day Car+"

# Create new column order
final_cols = [
    "joinkey", "week", "day", "ofd_date", "station", "cycle", "service_type",
    "value", "source", "flex_share_cvp", "flex_spr_cvp", "wave_capacity", "wave_start_time",
    "wave_frequency", "waves_available", "wave_max", "sa%_eoa", "sa%_week", "sa%_d3", "sa%_d2", "sa%_d1",
    "sa%_d0", "infinity_seq_mins", "seq_mins", "utr_buffer", "dea_cut_off", "dea_buffer_start",
    "flex_volume", "block_demand", "wave_demand", "waves_available", "D_1", "D_2", "D_3", "D_4",
    "D_5", "D_6", "D_7", "D_8", "D_9", "D_10", "D_11", "D_12", "D_13", "D_14", "D_15", "C_1", "C_2", "C_3",
    "C_4", "C_5", "C_6", "C_7", "C_8", "C_9", "C_10", "C_11", "C_12", "C_13", "C_14", "C_15",
    "infinity_block_length", "valid_from"
]

df = df[final_cols]

# Rename cols to readable names



df.query('D_1 >0').head()

C:\Users\jklas\AppData\Local\Temp\ipykernel_13324\1147735081.py:76: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df["infinity_seq_mins"][nd_group] = df["seq_mins"]
C:\Users\jklas\AppData\Local\Temp\ipykernel_13324\1147735081.py:76: SettingW

,joinkey,week,day,ofd_date,station,cycle,wave_capacity,wave_start_time,wave_frequency,wave_end_time,valid_from,wave_max,week_a,ofd_date_a,station_a,cycle_a,value,source,snop_fcst,co_fcst,flex_share_cvp,flex_spr_cvp,ofd_date_b,station_b,cycle_b,utr_buffer,calc_spr,sa%_eoa,sa%_week,sa%_d3,sa%_d2,sa%_d1,sa%_d0,ofd_date_e,station_e,cycle_e,D_1,D_2,D_3,D_4,D_5,D_6,D_7,D_8,D_9,D_10,D_11,D_12,D_13,D_14,D_15,C_1,C_2,C_3,C_4,C_5,C_6,C_7,C_8,C_9,C_10,C_11,C_12,C_13,C_14,C_15,waves_available,final_forecast,flex_volume,block_demand,wave_demand,seq_mins,infinity_seq_mins,dea_cut_off,dea_buffer_start,infinity_block_length,service_type
2233,2025-06-07DXN1CYCLE_SD_D,23,Sat,2025-06-07,DXN1,CYCLE_SD_D,20.0,19:30,15.0,20:00,2025-05-11,2.0,23,2025-06-07,DXN1,CYCLE_SD_D,450.0,snop_fcst,450.0,NaN,1.0,19.0,2025-06-07,DXN1,CYCLE_SD_D,20.0,19.0,NaN,NaN,NaN,NaN,NaN,NaN,2025-06-07,DXN1,CYCLE_SD_D,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19.0,19.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,450.0,450.0,23.684211,1.184211,100.0,NaN,NaN,NaN,NaN,AmFlex Same Day Car+
4774,2025-06-07DXM5CYCLE_SD_D,23,Sat,2025-06-07,DXM5,CYCLE_SD_D,26.0,19:15,15.0,20:00,2025-04-20,3.0,23,2025-06-07,DXM5,CYCLE_SD_D,1540.0,snop_fcst,1540.0,NaN,1.0,22.0,2025-06-07,DXM5,CYCLE_SD_D,5.0,22.0,0.0,0.45,0.5,0.700,0.800,0.900,2025-06-07,DXM5,CYCLE_SD_D,2.5,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,23.0,26.0,26.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3,1540.0,1540.0,70.000000,2.692308,175.0,115.0,NaN,NaN,NaN,AmFlex Same Day Car+
3097,2025-06-07DLS2CYCLE_SD_D,23,Sat,2025-06-07,DLS2,CYCLE_SD_D,20.0,19:15,15.0,20:00,2025-04-02,3.0,23,2025-06-07,DLS2,CYCLE_SD_D,1180.0,snop_fcst,1180.0,NaN,1.0,22.0,2025-06-07,DLS2,CYCLE_SD_D,15.0,22.0,0.0,0.45,0.5,0.700,0.800,0.925,2025-06-07,DLS2,CYCLE_SD_D,2.5,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18.0,20.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3,1180.0,1180.0,53.636364,2.681818,165.0,105.0,NaN,NaN,NaN,AmFlex Same Day Car+
1528,2025-06-07DIG1CYCLE_SD_D,23,Sat,2025-06-07,DIG1,CYCLE_SD_D,40.0,19:30,15.0,20:00,2025-04-23,2.0,23,2025-06-07,DIG1,CYCLE_SD_D,794.0,snop_fcst,794.0,NaN,1.0,17.0,2025-06-07,DIG1,CYCLE_SD_D,20.0,17.0,NaN,NaN,NaN,NaN,NaN,NaN,2025-06-07,DIG1,CYCLE_SD_D,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,38.0,38.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,794.0,794.0,46.705882,1.167647,100.0,NaN,NaN,NaN,NaN,AmFlex Same Day Car+
335,2025-06-07DHA2CYCLE_SD_D,23,Sat,2025-06-07,DHA2,CYCLE_SD_D,26.0,19:00,15.0,20:00,2025-04-02,4.0,23,2025-06-07,DHA2,CYCLE_SD_D,1668.0,snop_fcst,1668.0,NaN,1.0,22.0,2025-06-07,DHA2,CYCLE_SD_D,40.0,22.0,0.0,0.45,0.5,0.675,0.775,0.900,2025-06-07,DHA2,CYCLE_SD_D,2.5,2.5,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,23.0,23.0,26.0,26.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4,1668.0,1668.0,75.818182,2.916084,200.0,80.0,NaN,NaN,NaN,AmFlex Same Day Car+


In [ ]:

df.head()

,joinkey,week,day,ofd_date,station,cycle,service_type,value,source,flex_share_cvp,flex_spr_cvp,wave_capacity,wave_start_time,wave_frequency,waves_available,wave_max,sa%_eoa,sa%_week,sa%_d3,sa%_d2,sa%_d1,sa%_d0,infinity_seq_mins,seq_mins,utr_buffer,dea_cut_off,dea_buffer_start,flex_volume,block_demand,wave_demand,waves_available,D_1,D_2,D_3,D_4,D_5,D_6,D_7,D_8,D_9,D_10,D_11,D_12,D_13,D_14,D_15,C_1,C_2,C_3,C_4,C_5,C_6,C_7,C_8,C_9,C_10,C_11,C_12,C_13,C_14,C_15,infinity_block_length,valid_from
102,2025-06-08DXN1CYCLE_SD_D,23,Sun,2025-06-08,DXN1,CYCLE_SD_D,AmFlex Same Day Car+,469.0,snop_fcst,1.0,19.0,20.0,19:30,15.0,0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.0,20.0,NaN,NaN,469.0,24.684211,1.234211,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-05-11
4492,2025-06-08DXM5CYCLE_SD_D,23,Sun,2025-06-08,DXM5,CYCLE_SD_D,AmFlex Same Day Car+,1651.0,snop_fcst,1.0,22.0,26.0,19:15,15.0,0,3.0,0.0,0.45,0.5,0.7,0.8,0.900,-5.0,175.0,5.0,NaN,NaN,1651.0,75.045455,2.886364,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-04-20
3246,2025-06-08DNG2CYCLE_SD_D,23,Sun,2025-06-08,DNG2,CYCLE_SD_D,AmFlex Same Day Car+,750.0,snop_fcst,1.0,NaN,28.0,19:00,15.0,0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,750.0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-06-04
1672,2025-06-08DLS2CYCLE_SD_D,23,Sun,2025-06-08,DLS2,CYCLE_SD_D,AmFlex Same Day Car+,1236.0,snop_fcst,1.0,22.0,20.0,19:15,15.0,0,3.0,0.0,0.45,0.5,0.7,0.8,0.925,-15.0,165.0,15.0,NaN,NaN,1236.0,56.181818,2.809091,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-04-02
4530,2025-06-08DIG1CYCLE_SD_D,23,Sun,2025-06-08,DIG1,CYCLE_SD_D,AmFlex Same Day Car+,727.0,snop_fcst,1.0,17.0,40.0,19:30,15.0,0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.0,20.0,NaN,NaN,727.0,42.764706,1.069118,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-04-23


Upload Inputs to S3

In [133]:
buffer = BytesIO()
df.to_parquet(buffer)

client.put_object(
    Bucket=BUCKET,
    Key="scheduling_inputs.parquet",
    Body=buffer.getvalue(),
    ContentType="application/octet-stream"
)

print("✅ Uploaded scheduling_inputs.parquet")


✅ Uploaded scheduling_inputs.parquet


In [134]:
df = get_s3_object(BUCKET, "scheduling_inputs.parquet")

df.query('D_1.isna() == False').head()

,week,day,ofd_date,station,cycle,wave_capacity,wave_start_time,wave_frequency,wave_end_time,valid_from,wave_max,value,source,snop_fcst,co_fcst,flex_share_cvp,flex_spr_cvp,utr_buffer,calc_spr,sa%_eoa,sa%_week,sa%_d3,sa%_d2,sa%_d1,sa%_d0,D_1,D_2,D_3,D_4,D_5,D_6,D_7,D_8,D_9,D_10,D_11,D_12,D_13,D_14,D_15,C_1,C_2,C_3,C_4,C_5,C_6,C_7,C_8,C_9,C_10,C_11,C_12,C_13,C_14,C_15
2595,23,Sat,2025-06-07,DXN1,CYCLE_SD_D,20.0,19:30,15.0,20:00,2025-05-11,2.0,450.0,snop_fcst,450.0,NaN,1.0,19.0,20.0,19.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19.0,19.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1773,23,Sat,2025-06-07,DXM5,CYCLE_SD_D,26.0,19:15,15.0,20:00,2025-04-20,3.0,1540.0,snop_fcst,1540.0,NaN,1.0,23.0,5.0,23.0,0.0,0.45,0.5,0.700,0.800,0.900,2.5,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,23.0,26.0,26.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
425,23,Sat,2025-06-07,DLS2,CYCLE_SD_D,20.0,19:15,15.0,20:00,2025-04-02,3.0,1180.0,snop_fcst,1180.0,NaN,1.0,21.0,20.0,21.0,0.0,0.45,0.5,0.700,0.800,0.925,2.5,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18.0,20.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1100,23,Sat,2025-06-07,DIG1,CYCLE_SD_D,40.0,19:30,15.0,20:00,2025-04-23,2.0,794.0,snop_fcst,794.0,NaN,1.0,15.0,25.0,15.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,38.0,38.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3565,23,Sat,2025-06-07,DHA2,CYCLE_SD_D,26.0,19:00,15.0,20:00,2025-04-02,4.0,1668.0,snop_fcst,1668.0,NaN,1.0,22.0,40.0,22.0,0.0,0.45,0.5,0.675,0.775,0.900,2.5,2.5,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,23.0,23.0,26.0,26.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [124]:
df.columns

Index(['joinkey', 'week', 'day', 'ofd_date', 'station', 'cycle',
       'wave_capacity', 'wave_start_time', 'wave_frequency', 'wave_end_time',
       'valid_from', 'wave_max', 'value', 'source', 'snop_fcst', 'co_fcst',
       'flex_share_cvp', 'flex_spr_cvp', 'utr_buffer', 'calc_spr', 'sa%_eoa',
       'sa%_week', 'sa%_d3', 'sa%_d2', 'sa%_d1', 'sa%_d0', 'D_1', 'D_2', 'D_3',
       'D_4', 'D_5', 'D_6', 'D_7', 'D_8', 'D_9', 'D_10', 'D_11', 'D_12',
       'D_13', 'D_14', 'D_15', 'C_1', 'C_2', 'C_3', 'C_4', 'C_5', 'C_6', 'C_7',
       'C_8', 'C_9', 'C_10', 'C_11', 'C_12', 'C_13', 'C_14', 'C_15'],
      dtype='object')

In [125]:
bucket="uk-flex-scheduling"

print("Starting SA Table download")
df = get_s3_object(bucket, "UK_Flex_Schedule_Ahead_Percentage.xlsm")
print(df.head())

df = df.rename(columns={
    "DOW": "day",
    "Station": "station",
    "Cycle": "cycle",
    "SA% EOA": "sa%_eoa",
    "SA% WK": "sa%_week",
    "SA% D3": "sa%_d3",
    "SA% D2": "sa%_d2",
    "SA% D1": "sa%_d1",
    "SA% D0": "sa%_d0"
})

df = df.fillna(0)
sa_table = df.copy()

sa_table.head()

Starting SA Table download
   DOW Station    Cycle  SA% EOA  SA% WK  SA% D3  SA% D2  SA% D1  SA% D0
0  Sun    DAB1  CYCLE_1      0.9    0.45    0.50     0.7    0.85     NaN
1  Mon    DAB1  CYCLE_1      0.9    0.45    0.50     0.7    0.85     NaN
2  Tue    DAB1  CYCLE_1      0.9    0.40    0.45     0.6    0.75     NaN
3  Wed    DAB1  CYCLE_1      0.9    0.45    0.50     0.7    0.85     NaN
4  Thu    DAB1  CYCLE_1      0.9    0.45    0.50     0.7    0.85     NaN


,day,station,cycle,sa%_eoa,sa%_week,sa%_d3,sa%_d2,sa%_d1,sa%_d0
0,Sun,DAB1,CYCLE_1,0.9,0.45,0.50,0.7,0.85,0.0
1,Mon,DAB1,CYCLE_1,0.9,0.45,0.50,0.7,0.85,0.0
2,Tue,DAB1,CYCLE_1,0.9,0.40,0.45,0.6,0.75,0.0
3,Wed,DAB1,CYCLE_1,0.9,0.45,0.50,0.7,0.85,0.0
4,Thu,DAB1,CYCLE_1,0.9,0.45,0.50,0.7,0.85,0.0


In [ ]:
from utils.inputs_processing import InputsProcessor
from src.config import *

processor= InputsProcessor(
    bucket=scheduling_bucket,
    local_files=local_files
)

df = processor.process_sa_table()

df.head()

ModuleNotFoundError: No module named 'utils'